In [1]:
import pandas as pd
import ast
import numpy as np

In [64]:
def state_merge(path_to_data,path_to_new):
    df = pd.read_csv(path_to_data,index_col='Unnamed: 0')
    df['census'] = df['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
    df['county'] = df['census'].apply(lambda x: x[:5])
    df['state'] = df['county'].apply(lambda x: x[:2])
    state_df = df.groupby('state').mean()
    state_df=state_df[['comp','LILATracts_1And10']]
    state_df = state_df.join(state_data)
    #state_df.to_csv(path_to_new)
    return state_df

In [71]:
def county_merge(path_to_data,path_to_new):
    df = pd.read_csv(path_to_data,index_col='Unnamed: 0')
    df['census'] = df['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
    df['county'] = df['census'].apply(lambda x: x[:5])
    df['state'] = df['county'].apply(lambda x: x[:2])
    county_df = df.groupby('county').mean()
    county_df=county_df[['comp','LILATracts_1And10']]
    county_df = county_df.join(state_data)
    county_df.to_csv(path_to_new)

In [292]:
#read in target data
target = pd.read_csv('../data/target.csv', index_col=0)
target['CensusTract'] = target['CensusTract'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 

In [293]:
target['county'] = target['CensusTract'].apply(lambda x: x[:5]) 
target.set_index("county", inplace =True)
target = target.drop('CensusTract',axis=1)

In [297]:
county_desert = target.groupby('county').mean()

In [302]:
county_desert.reset_index(inplace=True)

In [349]:
target.index.unique()

Index(['01001', '01003', '01005', '01007', '01009', '01011', '01013', '01015',
       '01017', '01019',
       ...
       '56027', '56029', '56031', '56033', '56035', '56037', '56039', '56041',
       '56043', '56045'],
      dtype='object', name='county', length=3143)

### HEALTHY FOODS

In [24]:
healthy = pd.read_csv('../data/twitter_mongo/healthy_final.csv',index_col='Unnamed: 0')
healthy['census'] = healthy['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
healthy['county'] = healthy['census'].apply(lambda x: x[:5])
healthy['state'] = healthy['county'].apply(lambda x: x[:2])
#healthy.groupby('LILATracts_1And10').mean()['comp']

In [66]:
healthy[np.array([len(str(i))!=5 for i in healthy.county])]

,keyword,text,lat,long,census,comp,category,LILATracts_1And10,county,state


In [72]:
healthy.join()

,keyword,text,lat,long,census,comp,category,LILATracts_1And10,county,state
0,almond,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
1,avocado,Breakfast all day : On the Mix Slam with avoca...,25.752978,-80.261223,12086006206,-0.4389,healthy,0,12086,12
2,mango,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
3,raspberry,"Almond cake , mango and raspberry compote , le...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12
4,raspberry,"Dulcey , raspberry and coconut entremet at @ba...",25.754059,-80.260989,12086006206,0.0000,healthy,0,12086,12


In [68]:
county_sent = healthy.groupby('county').mean()

In [69]:
county_sent=county_sent[['comp','LILATracts_1And10']]

In [300]:
county_sent = county_sent.reset_index()

In [301]:
county_sent.head()

,county,comp,LILATracts_1And10
0,01003,0.265528,0.034483
1,01031,0.000000,0.000000
2,01065,0.000000,0.000000
3,01069,-0.127267,0.666667
4,01073,0.217862,0.500000


In [32]:
state_data = pd.read_csv('../data/state_data.csv',index_col=0)

In [162]:
county_data = pd.read_csv('../data/county_sent/county_data.csv',delimiter='\t')

In [397]:
county_data.head()

,FIPS,Name,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL


In [47]:
state_data['FIPS'] = state_data['FIPS'].apply(lambda x: "0"+str(x) if len(str(x))==1 else str(x)) 
state_data.set_index('FIPS',inplace=True)


In [49]:
merged_state = state_sent.join(state_data)

In [51]:
merged_state.to_csv('../data/state_sent.csv')

In [74]:
merged_state['healthy_sent']=merged_state['comp']

In [193]:
unhealthy = pd.read_csv('../data/twitter_mongo/ff_stores_final.csv',index_col='Unnamed: 0')
unhealthy['census'] = unhealthy['census'].apply(lambda x: "0"+str(x) if len(str(x))==10 else str(x)) 
unhealthy['county'] = unhealthy['census'].apply(lambda x: x[:5])
unhealthy['state'] = unhealthy['county'].apply(lambda x: x[:2])
state_unhealthy = unhealthy.groupby('county').mean()
state_unhealthy=state_unhealthy[['LILATracts_1And10','comp']]
state_unhealthy = state_unhealthy.reset_index()
#state_unhealthy = state_unhealthy.join(state_data)
#state_unhealthy.to_csv('../data/state_ff.csv')

In [194]:
state_unhealthy.head()

,county,comp
0,01001,0.000000
1,01003,0.341311
2,01015,0.000000
3,01031,0.390500
4,01043,0.140500


In [195]:
merged_county3 = pd.merge(state_unhealthy,merged_county2,how='right',left_on="county",right_on="FIPS")

In [196]:
merged_county3=merged_county3.drop('county',axis=1)

In [197]:
merged_county3['comp']=merged_county3['comp'].fillna(-2)

In [204]:
merged_county3.rename(columns={1:'grocery_sent'}, inplace=True)

In [206]:
merged_county3.columns=['ff_sent','grocery_sent','unhealthy_sent','healthy_sent','FIPS','Name','State']

In [276]:
merged_county3['final_sent']=final

In [340]:
merged_county3.shape

(3135, 10)

In [208]:
merged_county3[['healthy_sent','grocery_sent','unhealthy_sent','ff_sent']]=merged_county3[['healthy_sent','grocery_sent','unhealthy_sent','ff_sent']].apply(lambda x: round(x, 2))

In [228]:
for i in merged_county3.

ff_sent
grocery_sent
unhealthy_sent
healthy_sent
FIPS
Name
State
final_sent


In [229]:
first = merged_county3.unhealthy_sent==merged_county3.healthy_sent
second = merged_county3.unhealthy_sent>merged_county3.healthy_sent
third = merged_county3.unhealthy_sent<merged_county3.healthy_sent

In [239]:
fin = pd.DataFrame([first,second,third],index=["tie","unhealthy","healthy"]).T

In [240]:
fin['lowest_col'] = fin.idxmin(axis=1)

In [262]:
fin['healthy']=fin['healthy'].apply(int)

In [271]:
final=[]
for index, row in fin.iterrows():
    if row['tie']==1:
        final.append("tie")
    elif row['healthy']>row['unhealthy']:
        final.append("healthy")
    else:
        final.append("unhealthy")
    

In [274]:
fin = fin.drop('lowest_col',axis=1)

In [219]:
merged_county3['final_sent'] = merged_county3['final_sent'].apply(lambda x: "unhealthy" if x=="unhealthy_sent" else "healthy")

In [311]:
merged_county3=pd.merge(merged_county3,county_desert,how='left', left_on='FIPS',right_on='county')

In [312]:
merged_county3 = merged_county3.drop('county',axis=1)

In [369]:
np.argmax(merged_county2[['unhealthy_sent','healthy_sent']].values[1])

0

In [324]:
merged_county3.FIPS[0]

'01001'

In [317]:
merged_county3.to_csv('../data/county_sent/grouped_county_sent.csv')

In [112]:
merged_state3 = merged_state2.join(state_unhealthy)

In [113]:
merged_state3.fillna(merged_state3['comp'].mean(),inplace=True)

In [121]:
merged_state3.rename(columns={'comp':'ff_sent'}, inplace=True)

In [145]:
arr=np.array(['unhealthy','healthy'])
final = arr[(np.array(merged_state3['grocery_sent']+merged_state3['healthy_sent'])>(merged_state3['ff_sent']+merged_state3['unhealthy_sent']).values)*1]

In [123]:
max_val = merged_state3[['healthy_sent','grocery_sent','ff_sent','unhealthy_sent']].idxmax(axis=1)

In [125]:
merged_state3['max_sent']=max_val

In [146]:
merged_state3['final_sent']=final

In [147]:
merged_state3.to_csv('../data/grouped_state_sent.csv')

In [152]:
merged_state3[['healthy_sent','grocery_sent','unhealthy_sent','ff_sent']]=merged_state3[['healthy_sent','grocery_sent','unhealthy_sent','ff_sent']].apply(lambda x: round(x, 2))

In [389]:
merged_county2.head()

,grocery_sent,unhealthy_sent,healthy_sent,FIPS,Name,State
0,0.251700,0.256531,0.349901,04013,Maricopa,AZ
1,0.460450,0.251438,0.103511,04019,Pima,AZ
2,0.421500,0.182133,0.545983,04021,Pinal,AZ
3,0.263692,0.271055,0.191651,06001,Alameda,CA
4,0.075550,0.334719,0.221700,06013,Contra Costa,CA


In [391]:
final=[]
for index, row in merged_county2.iterrows():
    if row['healthy_sent']==row['unhealthy_sent']==-2.0:
        final.append("tie")
    elif row['healthy_sent']>row['unhealthy_sent']:
        final.append("healthy")
    else:
        final.append("unhealthy")
    

In [392]:
merged_county2['final_sent']=final

In [398]:
final_data = pd.merge(merged_county2,county_data,how='inner',left_on='FIPS',right_on='FIPS')

In [403]:
final_data[['grocery_sent','unhealthy_sent','healthy_sent']]=final_data[['grocery_sent','unhealthy_sent','healthy_sent']].apply(lambda x: round(x, 2))

In [422]:
unhealthy=[]
for i in final_data['unhealthy_sent']:
    if i == -2.0:
        unhealthy.append("no data")
    else:
        unhealthy.append(i)

In [419]:
len(final_data)

3232

In [424]:
final_data['healthy_sent']=healthy
final_data['unhealthy_sent']=unhealthy

In [329]:
fips=pd.read_csv('../data/county_sent/grouped_county_sent.csv',index_col=0)

In [330]:
fips.head()

,ff_sent,grocery_sent,unhealthy_sent,healthy_sent,FIPS,Name,State,final_sent,LILATracts_1And10_x,unhealthy1,healthy1,fips_full
0,0.00,-2.0,-2.00,-2.00,1001,Autauga,AL,tie,0.17,no data,no data,1001
1,0.34,-2.0,0.41,0.27,1003,Baldwin,AL,unhealthy,0.10,0.41,0.27,1003
2,0.00,-2.0,-2.00,-2.00,1015,Calhoun,AL,tie,0.23,no data,no data,1015
3,0.39,-2.0,0.36,0.00,1031,Coffee,AL,unhealthy,0.29,0.36,0,1031
4,0.14,-2.0,-2.00,-2.00,1043,Cullman,AL,tie,0.11,no data,no data,1043


In [331]:
fips['FIPS'] = fips['FIPS'].apply(lambda x: "0"+str(x) if len(str(x))==4 else str(x)) 


In [332]:
fips[np.array([len(str(i))=5 for i in fips.FIPS])]

,ff_sent,grocery_sent,unhealthy_sent,healthy_sent,FIPS,Name,State,final_sent,LILATracts_1And10_x,unhealthy1,healthy1,fips_full


In [425]:
final_data.to_csv('../data/county_sent/grouped_county_sent3.csv')

In [339]:
fips.shape

(3135, 12)

### UNHEALTHY FOODS

In [9]:
unhealthy = pd.read_csv('../data/twitter_mongo/unhealthy_final.csv',index_col='Unnamed: 0')

unhealthy.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.298508
1    0.310094
Name: comp, dtype: float64

### GROCERY STORES

In [10]:
grocery = pd.read_csv('../data/twitter_mongo/grocery_stores_final.csv',index_col='Unnamed: 0')

grocery.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.219386
1    0.432712
Name: comp, dtype: float64

In [52]:
unhealthy = pd.read_csv('../data/twitter_mongo/unhealthy_final.csv',index_col='Unnamed: 0')

### FAST FOODS

In [11]:
fastfood = pd.read_csv('../data/twitter_mongo/ff_stores_final.csv',index_col='Unnamed: 0')

fastfood.groupby('LILATracts_1And10').mean()['comp']

LILATracts_1And10
0    0.300494
1    0.344389
Name: comp, dtype: float64